In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/30.TecDoc'
else:
    print('fuck')

path_root

'.'

In [2]:
import pandas as pd

# = = =  = = =  = = =

import requests
import json

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

import time

In [3]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcyxVSA3TP7aNMfc5fgfxt'

In [4]:
directory_in = os.path.join(path_root, './file/2.part')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'crawler_1.xlsx'

In [5]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['Part Code'] = df_input['Part Code'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：916



,No,Brand,Product Group,Part Number,Aditional Description,GTIN/EAN,Packing Unit,Quantity Per Packaging Unit,Status,Remanufactured,Trade Number,Mandatory Material Certification,OEM,Picture,Url,Json_Src,Json_Information,Json_Criteria,Part Code
0,1,WESTLAKE,Clutch Kit,WBM001,3in1 Kit,6922145500013,1,3,Normal,False,,False,1223109;1223113;1223330;1223377;1223395;122354...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,"{""0"": ""https://digital-assets.tecalliance.serv...",{},"{""0"": {""\u00d8"": ""228 mm""}, ""1"": {""Weight"": ""4...",127680953
1,2,WESTLAKE,Clutch Kit,WBM002,3in1 Kit,6922145500020,1,3,Normal,False,,False,1204419;1204419.4;1207275;1208423;1223001;1223...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,"{""0"": ""https://digital-assets.tecalliance.serv...",{},"{""0"": {""\u00d8"": ""215 mm""}, ""1"": {""Weight"": ""5...",127680979
2,3,WESTLAKE,Clutch Kit,WBM003,2in1 Kit,6922145505629,1,2,Normal,False,,False,1223001;1223002;1223022;1223212;1223213;122321...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""\u00d8"": ""215 mm""}, ""1"": {""Weight"": ""5...",127680980
3,4,WESTLAKE,Clutch Kit,WBM004,3in1 Kit,6922145501119,1,3,Normal,False,,False,21207542691;21207561754,,https://web.tecalliance.net/tecdoc/qa/parts/45...,"{""0"": ""https://digital-assets.tecalliance.serv...",{},"{""0"": {""\u00d8"": ""200 mm""}, ""1"": {""Weight"": ""3...",127680982
4,5,WESTLAKE,Clutch Kit,WBM005,3in1 Kit,6922145506169,1,3,Normal,False,,False,21211223240,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""\u00d8"": ""230 mm""}, ""1"": {""Weight"": ""6...",153870776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,912,WESTLAKE,Clutch Kit,WVW158ZT,CSC kit,6922145512030,1,3,Normal,False,,False,03L141016N;03L141016S;04L141015;04L141015E;04L...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""Number of Teeth"": ""23""}, ""1"": {""\u00d8...",1367763499
912,913,WESTLAKE,Clutch Kit,WVW160SZ,DMF kit,6922145512146,1,4,Normal,False,,False,02A141165A;02A141165B;02A141165D;02A141165E;02...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""Number of Teeth"": ""28""}, ""1"": {""\u00d8...",1415094171
913,914,WESTLAKE,Clutch Kit,WVW161SZ,DMF kit,6922145512153,1,4,Normal,False,,False,02A141165A;02A141165B;02A141165D;02A141165E;02...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""Number of Teeth"": ""28""}, ""1"": {""\u00d8...",1415094207
914,915,WESTLAKE,Clutch Kit,WVW162SZ,DMF with CSC,6922145512658,1,4,Normal,False,,False,022141015S;022141016;02M141671A;02M141671B;03L...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,{},{},"{""0"": {""Number of Teeth"": ""23""}, ""1"": {""\u00d8...",1449328515


In [6]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                                                  1
 Brand                                                                        WESTLAKE
 Product Group                                                              Clutch Kit
 Part Number                                                                    WBM001
 Aditional Description                                                        3in1 Kit
 GTIN/EAN                                                                6922145500013
 Packing Unit                                                                        1
 Quantity Per Packaging Unit                                                         3
 Status                                                                         Normal
 Remanufactured                                                                  False
 Trade Number                                                                         
 Mandatory Material Certificati

In [7]:
data = {'getArticleLinkedAllLinkingTargetManufacturer2': {'articleCountry': 'DE',
                                                          'articleId': crawler_series['Part Code'],
                                                          'country': 'DE',
                                                          'countryGroupFlag': False,
                                                          'lang': 'qa',
                                                          'linkingTargetType': 'VLOB',
                                                          'provider': 23365}}

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'X-Api-Key': api}),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 200:
                break
            elif dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
    except requests.exceptions.ProxyError as e:
        if '456 CN Client Forbidden' in str(e):
            print('Long live the People\'s Republic of China!')
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'data': {'array': [{'manuId': 16, 'manuName': 'BMW'}]}, 'status': 200}

In [8]:
if type(dict_['data']) == str and dict_['data'].strip() == '':
    list_dict_make = []
elif type(dict_['data']) == dict:
    list_dict_make = dict_['data']['array']
else:
    print('fuck')

list_dict_make

[{'manuId': 16, 'manuName': 'BMW'}]

In [9]:
list_dict_correct.extend([{'Part Code': crawler_series['Part Code'],
                           'Make No': i + 1,
                           'Make': dict_make['manuName'].strip(),
                           'Make Code': dict_make['manuId'],
                           'JOIN_PCMC': f'''{crawler_series['Part Code']};{dict_make['manuId']}''',
                           'Check_Url': url_check} for i, dict_make in enumerate(list_dict_make)])

list_dict_correct

[{'Part Code': 127680953,
  'Make No': 1,
  'Make': 'BMW',
  'Make Code': 16,
  'JOIN_PCMC': '127680953;16',
  'Check_Url': 'https://web.tecalliance.net/tecdoc/qa/parts/4561/WBM001/detail?groups=479'}]

In [10]:
crawler_status = True

crawler_status

True

In [11]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                                  1
 Brand                                                                        WESTLAKE
 Product Group                                                              Clutch Kit
 Part Number                                                                    WBM001
 Aditional Description                                                        3in1 Kit
 GTIN/EAN                                                                6922145500013
 Packing Unit                                                                        1
 Quantity Per Packaging Unit                                                         3
 Status                                                                         Normal
 Remanufactured                                                                  False
 Trade Number                                                                         
 Mandatory Material Certification          

In [12]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, './test_make.xlsx'), index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,Part Code,Make No,Make,Make Code,JOIN_PCMC,Check_Url
0,127680953,1,BMW,16,127680953;16,https://web.tecalliance.net/tecdoc/qa/parts/45...


In [13]:
print('Output Finished !')
print()

Output Finished !



In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_make_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Brand,Product Group,Part Number,Aditional Description,GTIN/EAN,Packing Unit,Quantity Per Packaging Unit,Status,Remanufactured,Trade Number,Mandatory Material Certification,OEM,Picture,Url,Json_Src,Json_Information,Json_Criteria,Part Code,Check_Url
0,1,WESTLAKE,Clutch Kit,WBM001,3in1 Kit,6922145500013,1,3,Normal,False,,False,1223109;1223113;1223330;1223377;1223395;122354...,,https://web.tecalliance.net/tecdoc/qa/parts/45...,"{""0"": ""https://digital-assets.tecalliance.serv...",{},"{""0"": {""\u00d8"": ""228 mm""}, ""1"": {""Weight"": ""4...",127680953,https://web.tecalliance.net/tecdoc/qa/parts/45...
